In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

load_dotenv()

True

In [2]:
LLM = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    api_key=os.environ.get('OPEN_ROUTER_API_KEY'),
    base_url="https://openrouter.ai/api/v1",
)

In [3]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    score: int

In [4]:
def create_outline(state:BlogState):
    title = state['title']
    
    prompt = f'Generate an outline for blog on the topic {title}'
    
    outline = LLM.invoke(prompt).content
    
    state['outline'] = outline
    
    return state

In [5]:
def create_blog(state):
    title = state['title']
    outline = state['outline']
    
    prompt = f"Write a detailed proffesional blog on title {title}, outline {outline}"
    
    blog = LLM.invoke(prompt).content
    
    state['content'] = blog
    
    return state

In [6]:
def blog_eval(state):
    title = state['title']
    outline = state['outline']
    blog = state['content']
    
    prompt = f"""
         act as a  professional blog evaluator, Score my blog out of 10,
            Blog data title{title} outline {outline} content {blog}
    """
    score = LLM.invoke(prompt)
    state['score'] = score
    
    return state
    

In [7]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('blog_eval', blog_eval)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','blog_eval')
graph.add_edge('blog_eval',END)

workflow = graph.compile()

In [8]:
initial_state = {'title':'Need to Create an Indian opensource LLM models'}
final_state = workflow.invoke(initial_state)


In [9]:
print(final_state)

{'title': 'Need to Create an Indian opensource LLM models', 'outline': 'Here is a comprehensive outline for a blog post on the topic: **"Why India Needs to Create Its Own Open-Source LLMs."**\n\n---\n\n### **Blog Title Options**\n*   **Option 1 (Direct):** The Strategic Imperative: Why India Must Build Its Own Open-Source LLMs\n*   **Option 2 (Visionary):** Beyond Silicon Valley: The Case for an Indigenous AI Stack\n*   **Option 3 (Problem/Solution):** Decolonizing AI: Why India Needs an Open-Source Large Language Model\n\n---\n\n### **Blog Outline**\n\n#### **I. Introduction**\n*   **The Hook:** Start with the current global AI landscape—dominated by US tech giants (OpenAI, Google, Anthropic).\n*   **The Context:** Acknowledge the convenience of existing models but highlight the "black box" nature of closed systems.\n*   **The Thesis Statement:** Argue that for India’s digital sovereignty, cultural preservation, and economic growth, developing a robust, indigenous open-source LLM is n

In [10]:
print(final_state['score'])

content='**Blog Evaluator Report**\n\n**Blog Title:** Beyond Silicon Valley: The Strategic Imperative for India’s Own Open-Source LLM\n**Topic:** Indian AI Sovereignty / Open Source LLMs\n**Total Score:** **8.5 / 10**\n\n---\n\n### **Detailed Analysis**\n\n#### **1. Content Quality & Depth (9/10)**\n*   **Strengths:** The blog strikes an excellent balance between visionary rhetoric and concrete technical/economic realities. It successfully moves beyond generic "AI is cool" talking points to address specific Indian nuances (Hinglish, vernacular data, UPI context).\n*   **Argumentation:** The flow is logical. It moves from the problem (dependence) to the cultural imperative, then to sovereignty, economics, and finally execution. The distinction between "India" and "Bharat" regarding language models is particularly sharp and necessary.\n*   **Nuance:** The inclusion of "Challenges" (Section VII) elevates the piece from a wishful manifesto to a realistic strategic plan. Acknowledging the G